<a href="https://colab.research.google.com/github/KorStats/classification-project/blob/main/N%EB%B6%84%EB%A5%98_%EC%A6%9D%EB%B6%84%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 구글드라이브에서 파일 읽어오기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 산업분류자동화		 df_J_inc.csv	   E_inc_model.pt    'My Drive'
 A_inc_model.pt		 df_K_inc.csv	   F_ori_model.pt     N_spell_check.csv
 B_inc_model.pt		 df_M_inc.csv	   H_ori_model.pt     Q_ori_model.pt
 B_spellcheck.csv	 df_N_inc.csv	   H_spellcheck.csv   Q_spellcheck.csv
 classes.txt		'# df_P_inc.csv'   K_inc_model.pt     R_inc_model.pt
'Colab Notebooks'	 df_Q_inc.csv	   labels.zip	      S_ori_model.pt
'답안 작성용 파일.csv'	 df_R_inc.csv	   M_inc_model.pt     val.txt


In [ ]:
!unzip /content/gdrive/'My Drive'/산업분류자동화/'실습용 자료'.zip -d /content/dataset

Archive:  /content/gdrive/My Drive/산업분류자동화/실습용 자료.zip
1. 실습용자료.txt:  mismatching "local" filename (1. ьЛдьК╡ьЪйьЮРыгМ.txt),
         continuing with "central" filename version
  inflating: /content/dataset/1. 실습용자료.txt  


## 데이터 전처리

In [ ]:
import pandas as pd
df=pd.read_csv('/content/gdrive/MyDrive/N_spell_check.csv')

In [ ]:
df['digit_3'].value_counts()

751    3712
759    3583
752    2605
762    1948
742    1866
763    1482
761     938
741     783
753     524
743     239
764      21
Name: digit_3, dtype: int64

In [ ]:
# 데이터 증분 코드
import random
import pickle
import re
def get_only_hangul(line):
	parseText= re.compile('/ ^[ㄱ-ㅎㅏ-ㅣ가-힣]*$/').sub('',line)
	return parseText

def synonym_replacement(words, n):
	new_words = words.copy()
	random_word_list = list(set([word for word in words]))
	random.shuffle(random_word_list)
	num_replaced = 0
	for random_word in random_word_list:
		synonyms = get_synonyms(random_word)
		if len(synonyms) >= 1:
			synonym = random.choice(list(synonyms))
			new_words = [synonym if word == random_word else word for word in new_words]
			num_replaced += 1
		if num_replaced >= n:
			break
	if len(new_words) != 0:
		sentence = ' '.join(new_words)
		new_words = sentence.split(" ")
	else:
		new_words = ""
	return new_words

def get_synonyms(word):
	synomyms = []
	try:
		for syn in wordnet[word]:
			for s in syn:
				synomyms.append(s)
	except:
		pass
	return synomyms

def random_deletion(words, p):
	if len(words) == 1:
		return words

	new_words = []
	for word in words:
		r = random.uniform(0, 1)
		if r > p:
			new_words.append(word)
	if len(new_words) == 0:
		rand_int = random.randint(0, len(words)-1)
		return [words[rand_int]]
	return new_words

########################################################################
# Random swap
# Randomly swap two words in the sentence n times
########################################################################
def random_swap(words, n):
	new_words = words.copy()
	for _ in range(n):
		new_words = swap_word(new_words)
	return new_words

def swap_word(new_words):
	random_idx_1 = random.randint(0, len(new_words)-1)
	random_idx_2 = random_idx_1
	counter = 0
	while random_idx_2 == random_idx_1:
		random_idx_2 = random.randint(0, len(new_words)-1)
		counter += 1
		if counter > 3:
			return new_words
	new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1]
	return new_words

########################################################################
# Random insertion
# Randomly insert n words into the sentence
########################################################################
def random_insertion(words, n):
	new_words = words.copy()
	for _ in range(n):
		add_word(new_words)	
	return new_words

def add_word(new_words):
	synonyms = []
	counter = 0
	while len(synonyms) < 1:
		if len(new_words) >= 1:
			random_word = new_words[random.randint(0, len(new_words)-1)]
			synonyms = get_synonyms(random_word)
			counter += 1
		else:
			random_word = ""
		if counter >= 10:
			return
		
	random_synonym = synonyms[0]
	random_idx = random.randint(0, len(new_words)-1)
	new_words.insert(random_idx, random_synonym)

def EDA(sentence, alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.1, p_rd=0.1, num_aug=9):
	sentence = get_only_hangul(sentence)
	words = sentence.split(' ')
	words = [word for word in words if word is not ""]
	num_words = len(words)
	augmented_sentences = []
	num_new_per_technique = int(num_aug/4) + 1
	n_sr = max(1, int(alpha_sr*num_words))
	n_ri = max(1, int(alpha_ri*num_words))
	n_rs = max(1, int(alpha_rs*num_words))
	# sr
	for _ in range(num_new_per_technique):
		a_words = synonym_replacement(words, n_sr)
		augmented_sentences.append(' '.join(a_words))
	# ri
	for _ in range(num_new_per_technique):
		a_words = random_insertion(words, n_ri)
		augmented_sentences.append(' '.join(a_words))
	# rs
	for _ in range(num_new_per_technique):
		a_words = random_swap(words, n_rs)
		augmented_sentences.append(" ".join(a_words))
	# rd
	for _ in range(num_new_per_technique):
		a_words = random_deletion(words, p_rd)
		augmented_sentences.append(" ".join(a_words))
	augmented_sentences = [get_only_hangul(sentence) for sentence in augmented_sentences]
	random.shuffle(augmented_sentences)
	if num_aug >= 1:
		augmented_sentences = augmented_sentences[:num_aug]
	else:
		keep_prob = num_aug / len(augmented_sentences)
		augmented_sentences = [s for s in augmented_sentences if random.uniform(0, 1) < keep_prob]
	augmented_sentences.append(sentence)
	return augmented_sentences

In [ ]:
df1=df[df['digit_3']==751] 
df2=df[df['digit_3']==759] 
df3=df[df['digit_3']==752] 
df4=df[df['digit_3']==762] 
df5=df[df['digit_3']==742] 
df6=df[df['digit_3']==763]
df7=df[df['digit_3']==761]
df8=df[df['digit_3']==741]
df9=df[df['digit_3']==753]
df10=df[df['digit_3']==743]
df11=df[df['digit_3']==764]


df100=pd.concat([df1, df2, df3, df4, df5, df6])
df200=pd.concat([df7, df8, df9])
df300=pd.concat([df10, df11])

df100=df100.reset_index(drop=True)
df200=df200.reset_index(drop=True)
df300=df300.reset_index(drop=True)

In [ ]:
temp_result=[]
n=0
id_list=[]
digit_1_list=[]
digit_2_list=[]
digit_3_list=[]

for sentence in df200['text']:
  result_list=EDA(sentence, alpha_sr=0.0, alpha_ri=0.0, alpha_rs=0.5, p_rd=0.5, num_aug=2)
  id_list=['p'+str(df200['AI_id'][n]), 'p'+str(df200['AI_id'][n]), 'p'+str(df200['AI_id'][n])]
  digit_1_list=[df200['digit_1'][n], df200['digit_1'][n], df200['digit_1'][n]]
  digit_2_list=[df200['digit_2'][n], df200['digit_2'][n], df200['digit_2'][n]]
  digit_3_list=[df200['digit_3'][n], df200['digit_3'][n], df200['digit_3'][n]]
  data_to_insert={'AI_id' : id_list, 'digit_1' : digit_1_list, 'digit_2' : digit_2_list, 'digit_3': digit_3_list, 'text' : result_list}
  df200_to_insert = pd.DataFrame(data_to_insert)
  df200= df200.append(df200_to_insert, ignore_index=True)
  n=n+1

In [ ]:
df200['digit_3'].value_counts()

761    3752
741    3132
753    2096
Name: digit_3, dtype: int64

In [ ]:
df200=df200.reset_index(drop=True)

In [ ]:
temp_result=[]
n=0
id_list=[]
digit_1_list=[]
digit_2_list=[]
digit_3_list=[]

for sentence in df300['text']:
  result_list=EDA(sentence, alpha_sr=0.0, alpha_ri=0.0, alpha_rs=0.5, p_rd=0.5, num_aug=10)
  id_list=['p'+str(df300['AI_id'][n]), 'p'+str(df300['AI_id'][n]), 'p'+str(df300['AI_id'][n]), 'p'+str(df300['AI_id'][n]),'p'+str(df300['AI_id'][n]), 'p'+str(df300['AI_id'][n]), 'p'+str(df300['AI_id'][n]), 'p'+str(df300['AI_id'][n]),'p'+str(df300['AI_id'][n]), 'p'+str(df300['AI_id'][n]), 'p'+str(df300['AI_id'][n])]
  digit_1_list=[df300['digit_1'][n], df300['digit_1'][n], df300['digit_1'][n], df300['digit_1'][n],df300['digit_1'][n], df300['digit_1'][n], df300['digit_1'][n], df300['digit_1'][n],df300['digit_1'][n], df300['digit_1'][n], df300['digit_1'][n]]
  digit_2_list=[df300['digit_2'][n], df300['digit_2'][n], df300['digit_2'][n], df300['digit_2'][n],df300['digit_2'][n], df300['digit_2'][n], df300['digit_2'][n], df300['digit_2'][n],df300['digit_2'][n], df300['digit_2'][n], df300['digit_2'][n]]
  digit_3_list=[df300['digit_3'][n], df300['digit_3'][n], df300['digit_3'][n], df300['digit_3'][n],df300['digit_3'][n], df300['digit_3'][n], df300['digit_3'][n], df300['digit_3'][n],df300['digit_3'][n], df300['digit_3'][n], df300['digit_3'][n]]
  data_to_insert={'AI_id' : id_list, 'digit_1' : digit_1_list, 'digit_2' : digit_2_list, 'digit_3': digit_3_list, 'text' : result_list}
  df300_to_insert = pd.DataFrame(data_to_insert)
  df300= df300.append(df300_to_insert, ignore_index=True)
  n=n+1

In [ ]:
df300['digit_3'].value_counts()

743    2868
764     252
Name: digit_3, dtype: int64

기존 데이터 합치기

In [ ]:
df=pd.concat([df100, df200, df300])

In [ ]:
df['digit_3'].value_counts()

761    3752
751    3712
759    3583
741    3132
743    2868
752    2605
753    2096
762    1948
742    1866
763    1482
764     252
Name: digit_3, dtype: int64

In [ ]:
df.to_csv('df_N_inc.csv', index=False, encoding='utf-8-sig')